In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
import matplotlib.pyplot as plt

In [113]:
%matplotlib widget

In [114]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

# Checking for bugs

In [115]:
name_of_evil_file = "20180919_FC_LFP2_cen14_01_cc_01.res"
datapath = "/Users/jepe/scripting/cellpy/dev_data/bugfixing"

In [116]:
import os
import sys
import time
from pathlib import Path
print(f"running {sys.argv[0]}")

import cellpy
from cellpy import log
from cellpy import cellreader
from cellpy.parameters import prms

prms.Reader.use_cellpy_stat_file = False
#prms.Reader.cycle_mode = "cathode"
prms.Reader.sorted_data = False
log.setup_logging(default_level="INFO", custom_log_dir=os.getcwd())

filename = Path(datapath) / name_of_evil_file

assert os.path.isfile(filename)

running /Users/jepe/miniconda3/envs/cellpy/lib/python3.6/site-packages/ipykernel_launcher.py
(cellpy) - Filename: cellpy_errors.log
(cellpy) - Full path: /Users/jepe/scripting/cellpy/dev_utils/cellpy_errors.log
(cellpy) - Filename: cellpy_info.log
(cellpy) - Full path: /Users/jepe/scripting/cellpy/dev_utils/cellpy_info.log
(cellpy) - Filename: cellpy_debug.log
(cellpy) - Full path: /Users/jepe/scripting/cellpy/dev_utils/cellpy_debug.log


In [117]:
d = cellreader.CellpyData()
d.from_raw(filename)
d.set_mass(0.12)
d.make_step_table()
d.make_summary()

# checking extracting cycles
n = d.get_number_of_cycles()
c = d.get_cycle_numbers()

(cellpy) - created CellpyData instance


/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 11 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 22 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 33 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 44 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 55 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 66 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 77 is missing!
  if j + 1 not in unique_steps:
/Users/jepe/scripting/cellpy/cellpy/readers/cellreader.py:2960: UserWarning: Cycle 88 is m

In [118]:

current = d.dataset.dfdata
steps = d.dataset.step_table

In [119]:
steps.loc[(steps.step==9) | (steps.step==2), (steps.columns.str.startswith("cha")|steps.columns.str.startswith("ty"))]

,charge_avr,charge_std,charge_min,charge_max,charge_first,charge_last,charge_delta,type
1,0.000535,0.000571,1.321240e-10,0.001715,1.321240e-10,0.001715,1.298299e+09,charge
7,0.000402,0.000419,0.000000e+00,0.001238,0.000000e+00,0.001238,1.000000e+00,charge
13,0.000412,0.000395,6.302670e-10,0.001159,6.302670e-10,0.001159,1.838414e+08,charge
19,0.000406,0.000389,7.929460e-10,0.001151,7.929460e-10,0.001151,1.451555e+08,charge
25,0.000411,0.000391,6.302830e-10,0.001142,6.302830e-10,0.001142,1.811181e+08,charge
31,0.000403,0.000385,0.000000e+00,0.001140,0.000000e+00,0.001140,1.000000e+00,charge
37,0.000401,0.000384,0.000000e+00,0.001135,0.000000e+00,0.001135,1.000000e+00,charge
43,0.000402,0.000385,4.676310e-10,0.001130,4.676310e-10,0.001130,2.417424e+08,charge
49,0.000403,0.000385,0.000000e+00,0.001125,0.000000e+00,0.001125,1.000000e+00,charge
55,0.000403,0.000385,0.000000e+00,0.001123,0.000000e+00,0.001123,1.000000e+00,charge


In [120]:
steps.loc[(steps.cycle==2) & (steps.step==9)].T

,7
cycle,2
step,9
sub_step,1
point_avr,905
point_std,23.5266
point_min,865
point_max,945
point_first,865
point_last,945
point_delta,9.24855


charge_avr	0.000411908
charge_std	0.000394592
charge_min	6.30267e-10
charge_max	0.00115869
charge_first	6.30267e-10
charge_last	0.00115869
charge_delta	1.83841e+08



charge_avr	0.000402134
charge_std	0.000418774
charge_min	0
charge_max	0.001238
charge_first	0
charge_last	0.001238
charge_delta	
 
 
 

In [121]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
ax1.plot(current.Charge_Capacity, label="current")
ax1.plot(normal.Charge_Capacity, label="before", alpha=0.3)
ax1.legend()
ax2.plot(current.Step_Index)

FigureCanvasNbAgg()

In [122]:

c_current = current.loc[(current.Cycle_Index == 2)]

In [123]:
fig, ax = plt.subplots()
ax.plot(c_current.Voltage, '.r', label="current")
ax.legend()

FigureCanvasNbAgg()

In [7]:
dataset = d.dataset

In [124]:
normal = dataset.dfdata.copy()
steps = dataset.step_table.copy()
normal2 = normal.sort_values("Data_Point").reset_index()

In [125]:
c2_filter = (normal.Cycle_Index == 2) &  (normal.Step_Index == 9)
c3_filter = (normal.Cycle_Index == 3) &  (normal.Step_Index == 9)
c2 = normal.loc[c2_filter, ["Data_Point", "Step_Time", "Charge_Capacity"]]
c3 = normal.loc[c3_filter, ["Data_Point", "Step_Time", "Charge_Capacity"]]
c2b_filter = (normal2.Cycle_Index == 2) &  (normal2.Step_Index == 9)
c3b_filter = (normal2.Cycle_Index == 3) &  (normal2.Step_Index == 9)
c2b = normal2.loc[c2b_filter, ["Data_Point", "Step_Time", "Charge_Capacity"]]
c3b = normal2.loc[c3b_filter, ["Data_Point","Step_Time", "Charge_Capacity"]]

In [126]:
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.plot(c2.Step_Time, c2.Charge_Capacity, '-*')
ax1.plot(c2b.Step_Time, c2b.Charge_Capacity, '-ro')
ax2.plot(c2.Charge_Capacity)
ax2.plot(c2b.Charge_Capacity, 'r')
ax2.plot(c3b.Data_Point, c3b.Charge_Capacity, 'g')

FigureCanvasNbAgg()

In [132]:
v,c = d.get_cap()
#print(caps)
# checking creating dqdv

#d.to_csv(datapath, cycles=True)

In [137]:
fig, ax = plt.subplots(1)
ax.plot(v,c)

FigureCanvasNbAgg()